Filtro que a partir de un conjunto de valores repetidos, nos mostrará una lista con valores filtrados no repetidos. Eliminará los registros en base al dato del campo que le indiquemos. En este ejemplo eliminaremos los registros cuya dirección IP sea igual, ya que necesitamos un informe el usuario y las distintas direcciones IP desde las que se ha conectado.

In [1]:
%config IPCompleter.greedy=True

In [4]:
%%writefile valoresNoRepetidos.py
#!/usr/bin/env python

from mrjob.job import MRJob
import re

#Expresion regular para validar ip
ip = ('^(?:(?:25[0-5]|2[0-4][0-9]|'
          '[01]?[0-9][0-9]?)\.){3}'
          '(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$')
IP_RE = re.compile(ip)

class valoresNoRepetidos(MRJob):
    
    def mapper(self,_, line):
        linea=line.split(";")
        encontrado=re.search(IP_RE,linea[8])
        if encontrado!=None:    
            yield _,linea
        
    def reducer(self, key, values):
        listaDef=[] #Lista para los registros únicos filtrados
        listaControl=[] #Lista para el control de los registros que son repetidos
        
        # metemos los valores en las listas con lo que nos viene del mapper.
        for valor in values:
            #Si el valor de control (en este caso la IP) no está en la lista de control
            #agregamos el valor de control en la lista de control, y el registro completo en la 
            #lista definitiva.
            if valor[8] not in listaControl:
                listaControl.append(valor[8])
                listaDef.append(valor)
                
        #Podemos sacar por el reducer lo que estimemos        
        for p in listaDef:
            yield p[1], (p[1],p[8])
            
if __name__ == '__main__':
    valoresNoRepetidos.run()

Writing valoresNoRepetidos.py


In [5]:
!python valoresNoRepetidos.py archivos_datos/eventos_alumnos.csv

"Nieves Mar\u00eda Gonz\u00e1lez Fern\u00e1ndez"	["Nieves Mar\u00eda Gonz\u00e1lez Fern\u00e1ndez", "217.125.48.43"]
"Admin Manuel R."	["Admin Manuel R.", "87.221.75.73"]
"Gregorio Rodr\u00edguez Ortiz"	["Gregorio Rodr\u00edguez Ortiz", "195.77.157.196"]
"Mar\u00eda Patricia Barea Rold\u00e1n"	["Mar\u00eda Patricia Barea Rold\u00e1n", "37.222.51.156"]
"Manuel Jim\u00e9nez Guti\u00e9rrez"	["Manuel Jim\u00e9nez Guti\u00e9rrez", "31.4.245.175"]
"Jorge Vedia Molina"	["Jorge Vedia Molina", "87.223.30.0"]
"Mar\u00eda del Carmen P\u00e9rez Mu\u00f1oz"	["Mar\u00eda del Carmen P\u00e9rez Mu\u00f1oz", "85.61.123.29"]
"Andr\u00e9s Navarro Paule"	["Andr\u00e9s Navarro Paule", "80.39.209.6"]
"Francisco Javier Santiago Sanz"	["Francisco Javier Santiago Sanz", "81.32.157.225"]
"Jose Manuel Fuentes Quesada"	["Jose Manuel Fuentes Quesada", "150.214.205.80"]
"\u00c1ngel Ru\u00edz Jim\u00e9nez"	["\u00c1ngel Ru\u00edz Jim\u00e9nez", "176.83.39.22"]
"\u00c1ngel Ru\u00edz Jim\u00e9nez"	["\u00c1ngel Ru\u00ed

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidos.manuel.20201126.075655.021888
Running step 1 of 1...
job output is in C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidos.manuel.20201126.075655.021888\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidos.manuel.20201126.075655.021888\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidos.manuel.20201126.075655.021888...


In [1]:
%%writefile valoresNoRepetidosV2.py
#!/usr/bin/env python

from mrjob.job import MRJob
import re

#Expresion regular para validar ip
ip = ('^(?:(?:25[0-5]|2[0-4][0-9]|'
          '[01]?[0-9][0-9]?)\.){3}'
          '(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$')
IP_RE = re.compile(ip)

class valoresNoRepetidosV2(MRJob):
    
    def mapper(self,_, line):
        linea=line.split(";")
        encontrado=re.search(IP_RE,linea[8])
        if encontrado!=None:    
            yield linea[1],linea[8]
    
    def combiner(self,key,values):
        for valor in values:
            yield key, valor
    def reducer(self, key, values):
        listaDef=[] #Lista para los registros únicos filtrados
        listaControl=[] #Lista para el control de los registros que son repetidos
        
        # metemos los valores en las listas con lo que nos viene del mapper.
        for valor in values:
            #Si el valor de control (en este caso la IP) no está en la lista de control
            #agregamos el valor de control en la lista de control, y el registro completo en la 
            #lista definitiva.
            if valor not in listaControl:
                listaControl.append(valor)
                listaDef.append(valor)
                
        #Podemos sacar por el reducer lo que estimemos        
        for p in listaDef:
            yield key, p
            
if __name__ == '__main__':
    valoresNoRepetidosV2.run()

Writing valoresNoRepetidosV2.py


In [2]:
!python valoresNoRepetidosV2.py archivos_datos/eventos_alumnos.csv

"-"	"195.77.157.196"
"-"	"2.59.95.231"
"-"	"77.226.115.25"
"-"	"85.61.123.29"
"-"	"88.22.136.192"
"-"	"84.79.207.193"
"-"	"85.137.125.12"
"-"	"185.181.238.37"
"-"	"148.56.163.170"
"-"	"46.6.6.162"
"-"	"37.222.51.156"
"-"	"188.26.213.213"
"-"	"188.85.142.160"
"-"	"77.230.252.22"
"-"	"31.4.217.45"
"-"	"83.35.222.184"
"-"	"213.99.91.28"
"-"	"95.18.16.115"
"-"	"217.125.48.43"
"-"	"79.147.101.8"
"-"	"87.125.227.5"
"-"	"83.54.50.245"
"-"	"81.38.133.38"
"-"	"37.35.149.235"
"Admin Manuel R."	"87.221.75.73"
"Admin Manuel R."	"195.77.157.196"
"Admin Manuel R."	"87.221.73.119"
"Admin Manuel R."	"212.145.201.139"
"Ana Rodr\u00edguez Mart\u00edn"	"217.216.174.197"
"Ana Rodr\u00edguez Mart\u00edn"	"31.4.222.134"
"Ana Rodr\u00edguez Mart\u00edn"	"195.77.157.196"
"Ana Rodr\u00edguez Mart\u00edn"	"31.4.223.205"
"Ana Rodr\u00edguez Mart\u00edn"	"150.214.205.156"
"Ana Rodr\u00edguez Mart\u00edn"	"31.4.219.116"
"Ana Rodr\u00edguez Mart\u00edn"	"31.4.217.45"
"Andr\u00e9s Navarro Paule"	"80.39.209.6"
"Andr\

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidosV2.manuel.20201126.075620.832159
Running step 1 of 1...
job output is in C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidosV2.manuel.20201126.075620.832159\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidosV2.manuel.20201126.075620.832159\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\valoresNoRepetidosV2.manuel.20201126.075620.832159...
